In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import HUXt as H
import astropy.units as u
%matplotlib inline

In [2]:
def cone_cme_boundary_1d(r_boundary, longitude, time, v_boundary, cme):    

    # Center the longitude array on CME nose, running from -pi to pi, to avoid dealing with any 0/2pi crossings
    lon_cent = longitude - cme.longitude
    id_high = lon_cent > np.pi*u.rad
    lon_cent[id_high] = 2.0*np.pi*u.rad - lon_cent[id_high]
    id_low = lon_cent < -np.pi*u.rad
    lon_cent[id_low] = lon_cent[id_low] + 2.0*np.pi*u.rad 
    
    if (lon_cent >= -cme.width/2) & (lon_cent <= cme.width/2):
        # Longitude inside CME span.

        #  Compute y, the height of CME nose above the 30rS surface
        y = cme.v*(time - cme.t_launch)
        x = np.NaN*y.unit
        if (y >= 0*u.km) & (y < cme.radius):  # this is the front hemisphere of the spherical CME
            x = np.sqrt(y*(2*cme.radius - y))  # compute x, the distance of the current longitude from the nose
        elif (y >= (cme.radius + cme.thickness)) & (y <= (2*cme.radius + cme.thickness)):  # this is the back hemisphere of the spherical CME
            y = y - cme.thickness
            x = np.sqrt(y*(2*cme.radius - y))
        elif (cme.thickness > 0*u.km) & (y >= cme.radius) & (y <= (cme.radius + cme.thickness)):  # this is the "mass" between the hemispheres
            x = cme.radius
            
        theta = np.arctan(x / r_boundary)
        if (lon_cent >= - theta) & (lon_cent <= theta):
            v_boundary = cme.v
            
    return v_boundary


In [3]:
cme = H.ConeCME(t_launch=10000, longitude=40.0, width=30, v=1000, thickness=10)
cme_list_checked = [cme]
# Check only cone cmes in cme list 
model = H.HUXt1D(2000, lon=0.0, simtime=5.0, dt_scale=4.0)
buffersteps = np.fix( (5.0*u.day).to(u.s) / model.dt)
buffertime = buffersteps*model.dt
time = np.arange(-buffertime.value, model.simtime.value + model.dt.value, model.dt.value) * model.dt.unit

dlondt = model.twopi * model.dt / model.synodic_period
lon, dlon, Nlon = H.longitude_grid()

# How many radians of carrington rotation in this simulation length
simlon = model.twopi * model.simtime / model.synodic_period
# How many radians of carrington rotation in the spinup period
bufferlon = model.twopi * buffertime / model.synodic_period
# Find the carrigton longitude range spanned by the spinup and simulation period, centered on simulation longitude
lonint = np.arange(model.lon.value-bufferlon, model.lon.value + simlon+dlondt, dlondt)
# Rectify so that it is between 0 - 2pi
loninit = H._zerototwopi_(lonint)
# Interpolate the inner boundary speed to this higher resolution
vinit = np.interp(loninit, lon.value, model.v_boundary.value, period=model.twopi) * model.kms
# convert from cr longitude to time
vinput = np.flipud(vinit)

# ----------------------------------------------------------------------------------------
# Main model loop
# ----------------------------------------------------------------------------------------
iter_count = 0
i_out = 0
for i, t in enumerate(time):
            
    # Get the initial condition, which will update in the loop,
    # and snapshots saved to output at right steps.
    if i == 0:
        v_cme = np.ones(model.Nr)*400*model.kms
        v_amb = np.ones(model.Nr)*400*model.kms
    
    # Update the inner boundary conditions
    v_amb[0] = vinput[i]
    v_cme[0] = vinput[i]
    # Add in the cone CME at the boundary
    if t >= cme.t_launch:
        for c in cme_list_checked:
            r_boundary = model.r.min().to(u.km)
            v_update_cme = cone_cme_boundary_1d(r_boundary, model.lon, t, v_cme[0], c)
            
        v_cme[0] = v_update_cme
    
    # update cone cme v(r) for the given longitude
    # =====================================
    u_up = v_cme[1:].copy()
    u_dn = v_cme[:-1].copy()
    u_up_next = H._upwind_step_(model, u_up, u_dn)
    # Save the updated timestep
    v_cme[1:] = u_up_next.copy()

    u_up = v_amb[1:].copy()
    u_dn = v_amb[:-1].copy()
    u_up_next = H._upwind_step_(model, u_up, u_dn)
    # Save the updated timestep
    v_amb[1:] = u_up_next.copy()
    
    # Save this frame to output if output timestep is a factor of time elapsed 
    if t > 0:
        iter_count = iter_count + 1
        if iter_count == model.dt_scale.value:
            if i_out <= model.Nt_out - 1:
                model.v_grid_cme[i_out, :] = v_cme.copy()
                model.v_grid_amb[i_out, :] = v_amb.copy()
                i_out = i_out + 1
                iter_count = 0

In [4]:
model2 = H.HUXt1D(2000, lon=0.0, simtime=5.0, dt_scale=4.0)
model2.solve(cme_list_checked)

NameError: name 'cone_cme_boundary_1d' is not defined

In [ ]:
ii = 10
plt.plot(model.v_grid_cme[ii,:],'k-')
plt.plot(model2.v_grid_cme[ii,:],'r--')

In [ ]:
plt.plot(model.v_grid_cme[:, 50],'k-')
plt.plot(model2.v_grid_cme[:, 50],'r--')